# Step 10: Train feedforward regression model

After training a classification model, we now repeat the same process for the regression task. Once again, this notebook serves to prepare automatic testing of several network architectures. Results can be replicated using the corresponding Python script, logs are also stored in the repository.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import keras.backend as K
K.clear_session()

Using TensorFlow backend.


In [3]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)

## Prepare data

In [4]:
from tep.utils import load_array
feats = load_array("data/auxiliary_features.bc")
labels = load_array("data/regression_labels.bc")
print(feats.shape)
print(labels.shape)

(1293005, 15)
(1293005,)


In [5]:
# transform labels to log(x+1) in order to account for zero values
import numpy as np
print(labels[:20])
labels = np.log1p(labels)
print(labels[:20])

[ 120    0    3    5    0    3    0   46    2    0    0    1 2666   74    1
    3    0    2    0    0]
[ 4.79579055  0.          1.38629436  1.79175947  0.          1.38629436
  0.          3.8501476   1.09861229  0.          0.          0.69314718
  7.88870952  4.31748811  0.69314718  1.38629436  0.          1.09861229
  0.          0.        ]


## Configure model architectures

In [6]:
# define configurations to test
configs = [
    {'name': '1h_16n', 'num_layers': 1, 'num_units': 16},
    {'name': '1h_32n', 'num_layers': 1, 'num_units': 32},
    {'name': '1h_64n', 'num_layers': 1, 'num_units': 64},
    {'name': '2h_16n', 'num_layers': 2, 'num_units': 16},
    {'name': '2h_32n', 'num_layers': 2, 'num_units': 32},
    {'name': '2h_64n', 'num_layers': 2, 'num_units': 64},
    {'name': '3h_16n', 'num_layers': 3, 'num_units': 16},
    {'name': '3h_32n', 'num_layers': 3, 'num_units': 32},
    {'name': '3h_64n', 'num_layers': 3, 'num_units': 64},
    {'name': '4h_16n', 'num_layers': 4, 'num_units': 16},
    {'name': '4h_32n', 'num_layers': 4, 'num_units': 32},
    {'name': '4h_64n', 'num_layers': 4, 'num_units': 64},
]

## Train model

In [7]:
# use settings for testing on sample
train_size = 10000
val_size = 1000
batch_size = 64

# use settings for running on full data
#val_size = 10000
#train_size = features.shape[0] - val_size
#batch_size = 512

In [8]:
root_name = 'dffn_reg'
root_path = 'models/' + root_name

In [9]:
from tep.deepFeedforwardNetwork import regression_model
from tep.modelUtils import save_architecture
from tep.trainUtils import get_callbacks, print_regression_metrics, r2

for config in configs[:2]:
    # clear tf session first in order to avoid conflicts
    K.clear_session()

    # set model path
    model_name = root_name + '_' + config['name']
    model_path = root_path + '_' + config['name']
    logging_path = root_path + '/' + config['name']
    
    # Start logging
    print("Start training model: " + model_name)
    print("Training set: {} examples".format(train_size))
    print("Validation set: {} examples".format(val_size))
    
    # Create logging directory
    !mkdir -p $logging_path
    # Remove prior logs
    !rm $logging_path/*
    
    # load and save model
    model = regression_model(feats.shape[1], config['num_layers'], config['num_units'], metrics=[r2])
    save_architecture(model, model_path + '.json')
    
    # load model callbacks
    cbs = get_callbacks(model_name=model_name, log_dir=logging_path, verbose=1)
    
    # train model
    model.fit(feats[:train_size], 
          labels[:train_size], 
          validation_data=(feats[-val_size:], labels[-val_size:]), 
          batch_size=batch_size, 
          epochs=100, 
          verbose=0,
          shuffle=True,
          callbacks=cbs)
    
    # print best result
    history = cbs[2]
    print_regression_metrics(history)
    
    # add newline after model was trained
    print('\n')

Start training model: dffn_reg_1h_16n
Training set: 10000 examples
Validation set: 1000 examples
rm: models/dffn_reg/1h_16n/*: No such file or directory
Epoch 00001: val_loss improved from inf to 4.80259, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00002: val_loss improved from 4.80259 to 2.80228, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00003: val_loss improved from 2.80228 to 2.39926, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00004: val_loss improved from 2.39926 to 2.28721, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00005: val_loss improved from 2.28721 to 2.24948, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00006: val_loss improved from 2.24948 to 2.24068, saving model to models/dffn_reg_1h_16n.hdf5
Epoch 00007: val_loss did not improve
Epoch 00008: val_loss did not improve
Epoch 00009: val_loss did not improve
Epoch 00010: val_loss did not improve
Epoch 00011: val_loss improved from 2.24068 to 2.24025, saving model to models/dffn_reg_1h_16n.hdf5

## Analyze model performance

Results from running the training script suggest that the most suitable network architecture contains **5 layers** with **64 units** each. The model showed the following performance on unseen validation data:
* **loss: 1.0930**
* **R2: 0.7594**

Thanks to having saved weights and architecture, we can now load the model from disk and further analyze its regression performance.